In [1]:
# coding: utf-8
import pandas as pd
import numpy as np
from plutus.research.backtest.backtest import BacktestCS
from plutus.utils.visualization.plot import PlotCS

np.set_printoptions(suppress=True)
pd.set_option("display.float_format", lambda x: "%.5f" % x)

In [6]:
# 读取数据
datapath = "../../../datahub/raw/cn/stock/md/all_1d.parquet"
data_bfq = pd.read_parquet(datapath)
data_bfq.set_index(["trading_date", "code"], inplace=True)
data_bfq

turnover  down_limit   trade_num      low  \
trading_date code                                                       
2020-12-21   000001 1662649810.00000    16.52000 73755.00000 18.03000   
             000002 1692331342.00000    25.65000 66707.00000 27.97000   
             000004   46085820.00000    20.40000  4836.00000 22.50000   
             000005   16347044.00000     2.29000  1705.00000  2.52000   
             000006   27940502.00000     4.97000  1936.00000  5.44000   
...                              ...         ...         ...      ...   
2022-04-29   688799   17361423.00000    23.09000  1119.00000 28.88000   
             688800  151169883.00000    65.20000  5880.00000 83.05000   
             688819   51080384.00000    19.10000  5369.00000 24.16000   
             688981  711490492.00000    30.33000 33739.00000 38.43000   
             689009   99444025.00000    27.50000  6351.00000 34.00000   

                        high     open  up_limit         volume    close  \
trading_date code                                                         
2020-12-21   000001 18.64000 18.30000  20.20000 90687811.00000 18.28000   
             000002 28.43000 28.30000  31.35000 60094152.00000 28.17000   
             000004 23.15000 22.67000  24.94000  2017700.00000 22.72000   
             000005  2.57000  2.55000   2.79000  6422601.00000  2.55000   
             000006  5.52000  5.50000   6.07000  5092791.00000  5.49000   
...                      ...      ...       ...            ...      ...   
2022-04-29   688799 31.00000 29.39000  34.63000   573365.00000 30.61000   
             688800 97.80000 87.90000  97.80000  1675555.00000 97.80000   
             688819 25.43000 24.35000  28.64000  2040561.00000 25.39000   
             688981 39.99000 38.43000  45.49000 18076017.00000 39.72000   
             689009 38.10000 34.30000  41.26000  2735479.00000 37.18000   

                     pre_close  
trading_date code               
2020-12-21   000001   18.36000  
             000002   28.50000  
             000004   22.67000  
             000005    2.54000  
             000006    5.52000  
...                        ...  
2022-04-29   688799   28.86000  
             688800   81.50000  
             688819   23.87000  
             688981   37.91000  
             689009   34.38000  

[1457921 rows x 10 columns]

In [7]:
# 读取行业分类
datapath = "../../../datahub/raw/cn/stock/index/industry_sw1.parquet"
industry_sw1 = pd.read_parquet(datapath)
industry_sw1.set_index(["trading_date", "code"], inplace=True)
selected_industry_sw1 = industry_sw1.loc[industry_sw1["industry"] == "银行"]
selected_data_bfq = data_bfq.loc[selected_industry_sw1.index]
selected_data_bfq

turnover  down_limit   trade_num      low  \
trading_date code                                                       
2020-12-21   000001 1662649810.00000    16.52000 73755.00000 18.03000   
             002142 1262657341.00000    31.07000 51702.00000 34.00000   
             002807   66515862.00000     3.72000  5979.00000  4.10000   
             002839  138879017.00000     5.41000 11348.00000  5.96000   
             002936  137319297.00000     3.38000  9245.00000  3.71000   
...                              ...         ...         ...      ...   
2022-04-29   601963   61696999.00000     6.83000  9010.00000  7.51000   
             601988 1203757346.00000     2.93000 75979.00000  3.21000   
             601997  207605939.00000     5.48000 31958.00000  6.03000   
             601998  343610028.00000     4.51000 54210.00000  4.88000   
             603323  237290949.00000     4.78000 44141.00000  5.20000   

                        high     open  up_limit          volume    close  \
trading_date code                                                          
2020-12-21   000001 18.64000 18.30000  20.20000  90687811.00000 18.28000   
             002142 34.79000 34.52000  37.97000  36887930.00000 34.14000   
             002807  4.17000  4.13000   4.54000  16091380.00000  4.14000   
             002839  6.08000  6.01000   6.61000  23047878.00000  6.05000   
             002936  3.78000  3.74000   4.13000  36645012.00000  3.77000   
...                      ...      ...       ...             ...      ...   
2022-04-29   601963  7.74000  7.53000   8.35000   8062395.00000  7.73000   
             601988  3.27000  3.26000   3.59000 371435031.00000  3.23000   
             601997  6.18000  6.08000   6.70000  34004947.00000  6.15000   
             601998  5.09000  5.01000   5.51000  69469858.00000  5.03000   
             603323  5.42000  5.40000   5.84000  44782549.00000  5.28000   

                     pre_close  
trading_date code               
2020-12-21   000001   18.36000  
             002142   34.52000  
             002807    4.13000  
             002839    6.01000  
             002936    3.75000  
...                        ...  
2022-04-29   601963    7.59000  
             601988    3.26000  
             601997    6.09000  
             601998    5.01000  
             603323    5.31000  

[12998 rows x 10 columns]

In [ ]:
# 对数据进行基本的处理
open_ = data_bfq["open"].unstack()
close = data_bfq["close"].unstack()
high = data_bfq["high"].unstack()
low = data_bfq["low"].unstack()
vol = data_bfq["volume"].unstack()
amount = data_bfq["turnover"].unstack()

# 去除涨跌停,去除停牌股
# tradeable

# 获取基准
# benchmark
master_data = pd.DataFrame()
# #megedata["period"]=close.pct_change(1).shift(-1).stack()#以收盘价交易
master_data["period"] = open_.pct_change().shift(-2).stack()  # 以开盘价交易
master_data = master_data[np.isfinite(master_data).all(1)]

In [ ]:
# 定义一个因子
def factor_simple():
    factor = -1 * close.pct_change(5)
    return factor


test_factor = factor_simple()

In [ ]:
# 计算fator_rank
backtest_cs = BacktestCS()
clean_factor_data = backtest_cs.cal_factor_rank(master_data, test_factor)
clean_factor_data

In [ ]:
# 计算fator_quantile
clean_factor_data = backtest_cs.cal_factor_quantile(clean_factor_data, group_num=20)
clean_factor_data

In [ ]:
# 选择自己需要的hold_portfolio
long_portfolio_data, short_portfolio_data = backtest_cs.cal_hold_portfolio(
    clean_factor_data, hold_num=1
)
long_portfolio_data, short_portfolio_data

In [ ]:
# 计算标的权重,可以自己拟定
portfolio_data = backtest_cs.cal_portfolio_weight(long_portfolio_data)
portfolio_data

In [ ]:
# 计算回测指标
ret_df, sharpe_ratio, annual_return, max_down = backtest_cs.describer_01(
    long_portfolio_data, short_portfolio_data
)

In [ ]:
# 画出分组累计收益
PlotCS.plot_group_cumsum_pnl(clean_factor_data)

In [ ]:
# 画出long-short对冲收益
ret_df.plot(figsize=(16, 9), title="test")